In [ ]:
import pandas as pd
arquivo = pd.read_csv('C:/Users/Natan/OneDrive/Documentos/DidaticaTech/yelp.csv')
# Dataset: https://www.kaggle.com/omkarsabnis/yelp-reviews-dataset

In [ ]:
arquivo.head()

In [ ]:
arquivo.shape

In [ ]:
# Filtrando somente as colunas de interesse:
arquivo = arquivo[['text', 'stars']]

In [ ]:
arquivo.head()

In [ ]:
# Classificando ratings a partir de 4 como positivos e abaixo de 4 como negativos:
arquivo['sentimento']=['positivo' if (x>3) else 'negativo' for x in arquivo['stars']]

In [ ]:
arquivo.head()

In [ ]:
arquivo['sentimento'].value_counts()

In [ ]:
# Passando todas as palavras para lower case:
arquivo['text']= [x.lower() for x in arquivo['text']]

In [ ]:
# Vendo o resultado e observando que existem caracteres especiais:
pd.set_option('display.max_colwidth',None) # mostrando toda a largura de cada coluna, sem abreviar
arquivo.head(20)

In [ ]:
import re 

arquivo['text'] = arquivo['text'].apply((lambda x: re.sub('[^a-z0-9\s]','',x)))

In [ ]:
# Tokenizando as 3000 palavras mais frequentes:
from keras.preprocessing.text import Tokenizer
tokenizador = Tokenizer(num_words=3000) # se não informar num_words, ele irá considerar todas
tokenizador.fit_on_texts(arquivo['text'].values)

In [ ]:
# Visualizando as palavras mais frequentes
print(tokenizador.word_index)

In [ ]:
# O tokenizador foi criado, mas até aqui nosso dataset ainda não mudou. Iremos criar uma variável X com as palavras tokenizadas:
X = tokenizador.texts_to_sequences(arquivo['text'].values)

In [ ]:
X[2] # Observe como o texto do índice 2 possui 15 palavras

In [ ]:
# Verificando o tamanho do texto mais longo:
maximo_comprimento = max(len(i) for i in X)
print(maximo_comprimento)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Criando lotes de dados temporais, fazendo com que cada amostra tenha o mesmo tamanho:
X = pad_sequences(X)

In [ ]:
X[2]

In [ ]:
len(X[1])

In [ ]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(arquivo['sentimento']).values # Aplicando one-hot-encoding na variável target 
x_treino, x_teste, y_treino, y_teste = train_test_split(X,Y, test_size = 0.3)
print('Tamanho do treino: ', x_treino.shape, y_treino.shape)
print('Tamanho do teste: ', x_teste.shape, y_teste.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
import warnings
warnings.filterwarnings('ignore')

# Criando a rede neural temporal:

modelo = Sequential()
modelo.add(Embedding(input_dim=3000, output_dim=200, input_length = X.shape[1])) # criando os vetores word embedding. Lembrando que X.shape[1] é o número de palavras que cada amostra tem (907).
modelo.add(LSTM(150, dropout=0.3)) # criando a LSTM (150 neurônios na camada oculta)
modelo.add(Dense(2, activation='softmax')) # definindo a saída com 2 classes
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Treinando o modelo:
modelo.fit(x_treino, y_treino, batch_size = 500, epochs = 10, verbose = 2)

In [ ]:
# Testando a performance do modelo nos dados de teste:
__, acuracia = modelo.evaluate(x_teste, y_teste, verbose = 2, batch_size = 500)
print("Acurária com dados de teste: %.2f" % (acuracia))

In [ ]:
# Utilizando Word2Vec já treinado:
# pip install gensim
import gensim.models.keyedvectors as word2vec

In [ ]:
# Baixando o arquivo já treinado do Google News (1.5 GB!), depois de baixar, precisa descompactar:
# https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:
# Limitando a matriz embedding a 20.000 palavras mais frequentes treinadas:
modelo_word2vec = word2vec.KeyedVectors.load_word2vec_format('C:/Users/Natan/OneDrive/Documentos/DidaticaTech/GoogleNews-vectors-negative300.bin', binary=True, limit=20000)

In [ ]:
# Features:
modelo_word2vec.syn0.shape[1]

In [ ]:
# Tamanho vocabulário:
modelo_word2vec.syn0.shape[0]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
import warnings
warnings.filterwarnings('ignore')
# Criando a rede neural temporal:

modelo = Sequential()
modelo.add(Embedding(input_dim=modelo_word2vec.syn0.shape[0], output_dim=modelo_word2vec.syn0.shape[1], weights=[modelo_word2vec.syn0], 
                     input_length=X.shape[1])) # criando os vetores word embedding pré-treinados
modelo.add(LSTM(150, dropout=0.3)) # criando a LSTM (150 neurônios na camada oculta)
modelo.add(Dense(2, activation='softmax')) # definindo a saída com 2 classes
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Treinando o modelo:
modelo.fit(x_treino, y_treino, batch_size = 500, epochs = 10, verbose = 2)

In [ ]:
# Testando a performance do modelo nos dados de teste:
__, acuracia = modelo.evaluate(x_teste, y_teste, verbose = 2, batch_size = 500)
print("Acurária com dados de teste: %.2f" % (acuracia))